NYC Property Sales

Confirmed by Omar

This project aims to predict property sale prices in New York City using machine learning models. The dataset contains over 5 million real estate transactions with features like location, building type, square footage, and year built — ideal for GPU-accelerated regression.

Rows (Records): Over 5 million

Columns (Features): Around 20 column

In [ ]:
import cudf
import cuml
import cupy as cp
import matplotlib.pyplot as plt
import seaborn as sns
import time
import xgboost as xgb


from cuml.linear_model import LogisticRegression as cuLogisticRegression
from cuml.ensemble import RandomForestClassifier as cuRandomForestClassifier
from cuml.metrics import accuracy_score as cu_accuracy_score
from cuml.model_selection import train_test_split


In [ ]:
df = cudf.read_csv('/content/nyc-rolling-sales.csv')
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')


In [ ]:
for col in ['sale_price', 'land_square_feet', 'gross_square_feet']:
    df[col] = df[col].astype('str').str.replace('-', '').str.strip()
    df[col] = df[col].fillna('0').replace('', '0').astype('float32')

df['year_built'] = df['year_built'].fillna(0).astype('float32')
df['lot'] = df['lot'].fillna(0).astype('float32')
df['block'] = df['block'].fillna(0).astype('float32')

df = df[df['sale_price'] > 10000]
df = df.dropna()

In [ ]:
threshold = df['sale_price'].quantile(0.5)
df['high_value'] = (df['sale_price'] > threshold).astype('int32')

df['borough'] = df['borough'].astype('category').cat.codes
df['building_class_category'] = df['building_class_category'].astype('category').cat.codes
df['tax_class_at_time_of_sale'] = df['tax_class_at_time_of_sale'].astype('category').cat.codes

features = [
    'borough', 'lot', 'block', 'year_built',
    'tax_class_at_time_of_sale', 'building_class_category',
    'land_square_feet', 'gross_square_feet'
]
target = 'high_value'

X = df[features].astype('float32')
y = df[target].astype('int32')


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
start = time.time()

lr = cuLogisticRegression()
lr.fit(X_train, y_train)

lr_preds = lr.predict(X_test)
lr_acc = cu_accuracy_score(y_test, lr_preds)

elapsed = time.time() - start

print(f"✅ cuML Logistic Regression Accuracy: {float(lr_acc):.4f}")
print(f"⏱️ Training Time: {elapsed:.2f} seconds")


✅ cuML Logistic Regression Accuracy: 0.6569
⏱️ Training Time: 0.27 seconds


In [ ]:
start = time.time()

rf = cuRandomForestClassifier(n_estimators=100, max_depth=10)
rf.fit(X_train, y_train)

rf_preds = rf.predict(X_test)
rf_acc = cu_accuracy_score(y_test, rf_preds)

elapsed = time.time() - start

print(f"✅ cuML Random Forest Accuracy: {float(rf_acc):.4f}")
print(f"⏱️ Training Time: {elapsed:.2f} seconds")


✅ cuML Random Forest Accuracy: 0.8114
⏱️ Training Time: 0.49 seconds


## 📊 Additional Data Analysis

In [ ]:

# Check correlation between numerical variables
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap='coolwarm')
plt.title("Correlation Matrix")
plt.show()


KeyError: <class 'numpy.object_'>

<Figure size 1000x600 with 0 Axes>

In [ ]:

# Visualize distribution of sale prices
plt.figure(figsize=(8,5))
sns.histplot(df['sale_price'], bins=50, kde=True)
plt.title("Distribution of Sale Prices")
plt.xlabel("Sale Price")
plt.ylabel("Frequency")
plt.show()


## 🚀 Train XGBoost Model

In [ ]:

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Select features and target
features = [
    'borough', 'lot', 'block', 'year_built',
    'tax_class_at_time_of_sale', 'building_class_category',
    'land_square_feet', 'gross_square_feet'
]
target = 'high_value'

X = df[features]
y = df[target]

# Preprocess
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train XGBoost
xgb_model = XGBClassifier(n_estimators=100, max_depth=6, use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)

# Evaluate
y_pred = xgb_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"✅ XGBoost Model Accuracy: {accuracy:.4f}")


In [ ]:
import joblib

scaler = joblib.load("scaler.joblib")
lr_model = joblib.load("logistic_model.joblib")
rf_model = joblib.load("random_forest_model.joblib")
xgb_model = joblib.load("xgboost_gpu_model.joblib")


In [ ]:
joblib.dump(scaler, 'scaler.joblib')
joblib.dump(lr_model, 'logistic_model.joblib')
joblib.dump(rf_model, 'random_forest_model.joblib')
joblib.dump(xgb_model, 'xgboost_gpu_model.joblib')
